In [71]:
# import Modules

import pandas as pd
import numpy as np
import requests


In [72]:
# loading data source files from Google Drive
# loading file 1 "BYAREA1.txt"

URL = 'https://drive.google.com/file/d/1jhsJsA2jImF65qOXK_jWRmdF-GZgt4RF/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]

df1 = pd.read_csv(path, delimiter = '|', low_memory=False)


In [73]:
# loading data source files from Google Drive
# loading file 2 "BYAREA2.txt"

URL = 'https://drive.google.com/file/d/1dxLCHSdIaRwkOWYElxb7u37VhfWzY7Sp/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]

df2 = pd.read_csv(path, delimiter = '|', low_memory=False)


In [75]:
# combine two files into one df

data = pd.concat([df1, df2])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1145664 entries, 0 to 522271
Data columns (total 14 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   AREA                   1145664 non-null  object
 1   AGE_ADJUSTED_CI_LOWER  1145664 non-null  object
 2   AGE_ADJUSTED_CI_UPPER  1145664 non-null  object
 3   AGE_ADJUSTED_RATE      1145664 non-null  object
 4   COUNT                  1145664 non-null  object
 5   EVENT_TYPE             1145664 non-null  object
 6   POPULATION             1145664 non-null  int64 
 7   RACE                   1145664 non-null  object
 8   SEX                    1145664 non-null  object
 9   SITE                   1145664 non-null  object
 10  YEAR                   1145664 non-null  object
 11  CRUDE_CI_LOWER         1145664 non-null  object
 12  CRUDE_CI_UPPER         1145664 non-null  object
 13  CRUDE_RATE             1145664 non-null  object
dtypes: int64(1), object(13)
memory usag

In [76]:
# extract columns needed for my analysis

df = data[['AREA', 'COUNT', 'EVENT_TYPE', 'POPULATION', 'RACE', 'SEX', 'SITE', 'YEAR']]
df.head()

,AREA,COUNT,EVENT_TYPE,POPULATION,RACE,SEX,SITE,YEAR
0,Alabama,9299,Incidence,2293259,All Races,Female,All Cancer Sites Combined,1999
1,Alabama,4366,Mortality,2293259,All Races,Female,All Cancer Sites Combined,1999
2,Alabama,9474,Incidence,2302835,All Races,Female,All Cancer Sites Combined,2000
3,Alabama,4425,Mortality,2302835,All Races,Female,All Cancer Sites Combined,2000
4,Alabama,9971,Incidence,2309496,All Races,Female,All Cancer Sites Combined,2001


In [77]:
# removing "Mortality" from the data as I will be concentrating on "Incidence"

df = df[df['EVENT_TYPE'] == 'Incidence']
df.head()

,AREA,COUNT,EVENT_TYPE,POPULATION,RACE,SEX,SITE,YEAR
0,Alabama,9299,Incidence,2293259,All Races,Female,All Cancer Sites Combined,1999
2,Alabama,9474,Incidence,2302835,All Races,Female,All Cancer Sites Combined,2000
4,Alabama,9971,Incidence,2309496,All Races,Female,All Cancer Sites Combined,2001
6,Alabama,10151,Incidence,2314370,All Races,Female,All Cancer Sites Combined,2002
8,Alabama,9595,Incidence,2324069,All Races,Female,All Cancer Sites Combined,2003


In [80]:
# keeping summary lines like 'All Cancers', 'All Races', and 'Both Genders'

summary_df = df[(df['SITE'] == 'All Cancer Sites Combined') | (df['RACE'] == 'All Races') | (df['SEX'] == 'Male and Female') | (df['YEAR'] == '2013-2017')]


In [81]:
# removing summary lines, such as All Cancer Types, All Races, and Both Genders.

data = df[~((df['SITE'] == 'All Cancer Sites Combined') | (df['RACE'] == 'All Races') | (df['SEX'] == 'Male and Female'))]
data.head()

,AREA,COUNT,EVENT_TYPE,POPULATION,RACE,SEX,SITE,YEAR
2900,Alabama,~,Incidence,11411,American Indian/Alaska Native,Female,Brain and Other Nervous System,1999
2902,Alabama,~,Incidence,12506,American Indian/Alaska Native,Female,Brain and Other Nervous System,2000
2904,Alabama,~,Incidence,12933,American Indian/Alaska Native,Female,Brain and Other Nervous System,2001
2906,Alabama,~,Incidence,13377,American Indian/Alaska Native,Female,Brain and Other Nervous System,2002
2908,Alabama,~,Incidence,13863,American Indian/Alaska Native,Female,Brain and Other Nervous System,2003


In [83]:
# removing summary line for YEAR which has format like '2001-2007'

df = data[data['YEAR'].apply(lambda x: len(str(x)) == 4)]


In [84]:
# rename columns

df.columns = ['STATE', 'COUNT', 'EVENT_TYPE', 'POPULATION', 'RACE', 'GENDER', 'CANCER TYPE', 'YEAR']
df.head()

,STATE,COUNT,EVENT_TYPE,POPULATION,RACE,GENDER,CANCER TYPE,YEAR
2900,Alabama,~,Incidence,11411,American Indian/Alaska Native,Female,Brain and Other Nervous System,1999
2902,Alabama,~,Incidence,12506,American Indian/Alaska Native,Female,Brain and Other Nervous System,2000
2904,Alabama,~,Incidence,12933,American Indian/Alaska Native,Female,Brain and Other Nervous System,2001
2906,Alabama,~,Incidence,13377,American Indian/Alaska Native,Female,Brain and Other Nervous System,2002
2908,Alabama,~,Incidence,13863,American Indian/Alaska Native,Female,Brain and Other Nervous System,2003


In [85]:
# dropping rows with its 'COUNT' value being non-numerical
# converting COUNT into integer type

filter = df['COUNT'].str.isnumeric() == True
df1 = df[filter]

df1['COUNT'] = df1['COUNT'].apply(pd.to_numeric)
#df1['YEAR'] = df1['YEAR'].apply(pd.to_datetime, format='%Y')

df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111815 entries, 5960 to 521230
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   STATE        111815 non-null  object
 1   COUNT        111815 non-null  int64 
 2   EVENT_TYPE   111815 non-null  object
 3   POPULATION   111815 non-null  int64 
 4   RACE         111815 non-null  object
 5   GENDER       111815 non-null  object
 6   CANCER TYPE  111815 non-null  object
 7   YEAR         111815 non-null  object
dtypes: int64(2), object(6)
memory usage: 7.7+ MB


<ipython-input-85-b5f0d35d27ab>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['COUNT'] = df1['COUNT'].apply(pd.to_numeric)


In [86]:
# removing STATE from the data to get data for all US

df_nostate = df1[['YEAR','CANCER TYPE', 'RACE', 'GENDER', 'COUNT']]
df_nostate.head()


,YEAR,CANCER TYPE,RACE,GENDER,COUNT
5960,1999,Female Breast,Asian/Pacific Islander,Female,18
5964,2001,Female Breast,Asian/Pacific Islander,Female,16
5968,2003,Female Breast,Asian/Pacific Islander,Female,17
5974,2006,Female Breast,Asian/Pacific Islander,Female,20
5978,2008,Female Breast,Asian/Pacific Islander,Female,23


In [87]:
# summarize cancer statistic by year, cancer type, race, and gender.

df_sum = df_nostate.groupby(['YEAR', 'CANCER TYPE', 'RACE', 'GENDER']).sum()

df_sum['TOTAL (by Year & Cancer Type)'] = df_sum['COUNT'].groupby(['YEAR', 'CANCER TYPE']).sum()
df_sum['%'] = df_sum['COUNT'] / df_sum['TOTAL (by Year & Cancer Type)'] * 100

df_sum.head(30)


COUNT  \
YEAR CANCER TYPE                    RACE                          GENDER           
1999 Brain and Other Nervous System American Indian/Alaska Native Female      19   
                                                                  Male        22   
                                    Asian/Pacific Islander        Female     212   
                                                                  Male       246   
                                    Black                         Female     962   
                                                                  Male      1059   
                                    Hispanic                      Female    1323   
                                                                  Male      1536   
                                    White                         Female   14014   
                                                                  Male     16656   
     Cervix                         American Indian/Alaska Native Female     181   
                                    Asian/Pacific Islander        Female     891   
                                    Black                         Female    4733   
                                    Hispanic                      Female    4626   
                                    White                         Female   18343   
     Colon and Rectum               American Indian/Alaska Native Female     648   
                                                                  Male       770   
                                    Asian/Pacific Islander        Female    4264   
                                                                  Male      4363   
                                    Black                         Female   14404   
                                                                  Male     12486   
                                    Hispanic                      Female    7920   
                                                                  Male      8366   
                                    White                         Female  147603   
                                                                  Male    152121   
     Corpus and Uterus, NOS         American Indian/Alaska Native Female     273   
                                    Asian/Pacific Islander        Female    2049   
                                    Black                         Female    5917   
                                    Hispanic                      Female    4147   
                                    White                         Female   72856   

                                                                          TOTAL (by Year & Cancer Type)  \
YEAR CANCER TYPE                    RACE                          GENDER                                  
1999 Brain and Other Nervous System American Indian/Alaska Native Female                          36049   
                                                                  Male                            36049   
                                    Asian/Pacific Islander        Female                          36049   
                                                                  Male                            36049   
                                    Black                         Female                          36049   
                                                                  Male                            36049   
                                    Hispanic                      Female                          36049   
                                                                  Male                            36049   
                                    White                         Female                          36049   
                                                                  Male                            36049   
     Cervix                         American Indian/Alaska Native Female                          2877